# Encrypted Linear Regression

In this tutorial you are going to see how you can run a linear regression model on **data distributed in a pool of workers** with **encrypted computations leveraged by Secured Multi-Party Computation**. For this demonstration we are going to use the classical Housing Prices dataset that is already available in the VirtualGrid set up by the Syft Sandbox.

The idea for the implementation of the Encrypted Linear Regression algorithm in PySyft is based on the section 2 of [this paper](https://arxiv.org/abs/1901.09531) written by Jonathan Bloom of the Broad Institute of MIT and Harvard.

**Author**: André Macedo Farias. Github: [@andrelmfarias](https://github.com/andrelmfarias) | Twitter: [@andrelmfarias](https://twitter.com/andrelmfarias)

## 1. Preliminaries

First, let's import PySyft and PyTorch and set up the Syft sandbox, which will create all the objects and tools we will need to run our simulation (Virtual Workers, VirtualGrid with datasets, etc...)

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import torch
import syft as sy
sy.create_sandbox(globals(), verbose=False)

Setting up Sandbox...
Done!


You can see that we have several workers already set up:

In [3]:
workers

[<VirtualWorker id:bob #objects:14>,
 <VirtualWorker id:theo #objects:14>,
 <VirtualWorker id:jason #objects:14>,
 <VirtualWorker id:alice #objects:14>,
 <VirtualWorker id:andy #objects:14>,
 <VirtualWorker id:jon #objects:14>]

And each one has a chunk of the Housing Prices dataset:

In [4]:
for worker in workers:
    print(worker.search(["#housing", "#data"]))

[(Wrapper)>[PointerTensor | me:30189164229 -> bob:89440057589]
	Tags: #data #boston .. _boston_dataset: #boston_housing #housing 
	Shape: torch.Size([84, 13])
	Description: .. _boston_dataset:...]
[(Wrapper)>[PointerTensor | me:8249706715 -> theo:46905611490]
	Tags: #data #boston .. _boston_dataset: #boston_housing #housing 
	Shape: torch.Size([84, 13])
	Description: .. _boston_dataset:...]
[(Wrapper)>[PointerTensor | me:40521198768 -> jason:18847732952]
	Tags: #data #boston .. _boston_dataset: #boston_housing #housing 
	Shape: torch.Size([84, 13])
	Description: .. _boston_dataset:...]
[(Wrapper)>[PointerTensor | me:38604677687 -> alice:85568866295]
	Tags: #data #boston .. _boston_dataset: #boston_housing #housing 
	Shape: torch.Size([84, 13])
	Description: .. _boston_dataset:...]
[(Wrapper)>[PointerTensor | me:84600693994 -> andy:27396108387]
	Tags: #data #boston .. _boston_dataset: #boston_housing #housing 
	Shape: torch.Size([84, 13])
	Description: .. _boston_dataset:...]
[(Wrapper)

## 2. Encrypted Linear Regression with PySyft 

### 2.1 Loading Housing Prices data from Virtual Grid

Now we have our Syft environment set, let's load the data.

Please note that in order to avoid overflow with the SMPC computations performed by the linear model, and to maintain its stability, **we need to scale the data in a such way that the magnitude of each coordinate average lies in the interval [0.1, 10]**.

Usually that can be done without revealing the data or the averages, you only need to have an idea of the order of magnitude. For example, if one of the coordinate is the surface of the house and it is represented in m², you should scale it by dividing by 100, as we know the surfaces of houses have an order of magnitude close to 100 in average.

After running the model and obtaining the main statistics, we can rescale them back if needed. The same can be done with predictions.

In this tutorial I will be loading the data and scale them following this idea:

In [5]:
scale_data = torch.Tensor([10., 10.,  10., 1., 1., 10., 100., 10., 10., 1000., 10., 1000., 10.])
scale_target = 100.0

housing_data = []
housing_targets = []
for worker in workers:
    housing_data.append(worker.search(["#housing", "#data"])[0] / scale_data.send(worker))
    housing_targets.append(worker.search(["#housing", "#target"])[0] / scale_target)

### 2.2 Setting up 2 more Virtual workers: the crypto provider and the "honest but curious" worker

In order to run the linear regression, we will need **two more workers**, a *crypto provider* and a *honest but curious* worker. Both are necessary to assure the security of the SMPC computations when we run the model in a pool with more than 3 workers.

> *Note: the **honest but curious** worker is a legitimate participant in a communication protocol who will not deviate from the defined protocol but will attempt to learn all possible information from legitimately received messages.*

In [6]:
crypto_prov = sy.VirtualWorker(hook, id="crypto_prov")
hbc_worker = sy.VirtualWorker(hook, id="hbc_worker")

### 2.3 Running Encrypted Linear Regression with SMPC

Now let's import the EncryptedLinearRegression from the linalg module of pysyft:

In [7]:
from syft.frameworks.torch.linalg import EncryptedLinearRegression

Let's train the model!!

In [8]:
crypto_lr = EncryptedLinearRegression(crypto_provider=crypto_prov, hbc_worker=hbc_worker)
crypto_lr.fit(housing_data, housing_targets)

We can display the results with the method `.summarize()`

In [9]:
crypto_lr.summarize()

           SMPC Linear Regression Results
                 value         stderr        p-value
----------------------------------------------------
coef1          -0.0108         0.0033         0.0011
coef2           0.0046         0.0014         0.0008
coef3           0.0020         0.0062         0.7452
coef4           0.0269         0.0086         0.0019
coef5          -0.1775         0.0382         0.0000
coef6           0.3807         0.0418         0.0000
coef7           0.0008         0.0132         0.9541
coef8          -0.1474         0.0200         0.0000
coef9           0.0305         0.0066         0.0000
coef10         -0.1228         0.0376         0.0012
coef11         -0.0953         0.0131         0.0000
coef12          0.0930         0.0269         0.0006
coef13         -0.0525         0.0051         0.0000
intercept       0.3645         0.0511         0.0000
----------------------------------------------------


**We can see that the EncryptedLinearRegression does not only give the coefficients and intercept values, but also their standard errors and the p-values!**

## 3. Comparing results with other linear regressors

Now, in order to show the effectiveness of the EncryptedLinearRegression, let's compare it with the Linear Regression from other known libraries.

### 3.1 Sending data to local server for comparison purposes

First, let's send the data to the local worker and transform the `torch.Tensor`s in `numpy.array`s

In [10]:
import numpy as np

data_tensors = [x.copy().get() for x in housing_data] 
target_tensors = [y.copy().get() for y in housing_targets]

data_np = torch.cat(data_tensors, dim=0).numpy()
target_np = torch.cat(target_tensors, dim=0).numpy()

### 3.2 Scikit-learn

First let's compare the results with the sklearn's Linear Regression:

In [11]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression().fit(data_np, target_np.squeeze())

Display the results:

In [12]:
print("=" * 25)
print("Sklearn Linear Regression")
print("=" * 25)
for i, coef in enumerate(lr.coef_, 1):
    print(" coeff{:<3d}".format(i), "{:>14.4f}".format(coef))
print(" intercept:", "{:>12.4f}".format(lr.intercept_))
print("=" * 25)

Sklearn Linear Regression
 coeff1          -0.0108
 coeff2           0.0046
 coeff3           0.0021
 coeff4           0.0269
 coeff5          -0.1777
 coeff6           0.3810
 coeff7           0.0007
 coeff8          -0.1476
 coeff9           0.0306
 coeff10         -0.1233
 coeff11         -0.0953
 coeff12          0.0931
 coeff13         -0.0525
 intercept:       0.3646


**You can notice that the are results are pretty much the same!! The are some small differences, but they are never higher than 0.2% of the value computed by the sklearn model!!**

**For an ecrypted model that can compute linear regression coefficients without ever revealing the data, this is a huge achievement!**

### 3.3 Statsmodel API

We can do the same using the Linear Regression from Statsmodel API, which also gives us the **standard errors** and **p-values** of the coefficients. We can then compare it with the results given by the EncryptedLinearRegression

In [13]:
import statsmodels.api as sm
mod = sm.OLS(target_np.squeeze(), sm.add_constant(data_np), hasconst=True)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.741
Model:                            OLS   Adj. R-squared:                  0.734
Method:                 Least Squares   F-statistic:                     108.1
Date:                Fri, 11 Oct 2019   Prob (F-statistic):          6.72e-135
Time:                        15:46:06   Log-Likelihood:                 831.41
No. Observations:                 506   AIC:                            -1635.
Df Residuals:                     492   BIC:                            -1576.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3646      0.051      7.144      0.0

**Once again, we can see that all results are pretty much the same!!**



## Well Done!

And voilà! We were able to train an OLS Regression model on distributed data and without ever seeing it. We were even able to compute standard errors and p-values for each coefficient.

Also, after comparing our results with results given by other known libraries, we were able to validate this approach.

# Congratulations!!! - Time to Join the Community!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways!


### Star PySyft on GitHub

The easiest way to help our community is just by starring the repositories! This helps raise awareness of the cool tools we're building.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Pick our tutorials on GitHub!

We made really nice tutorials to get a better understanding of what Federated and Privacy-Preserving Learning should look like and how we are building the bricks for this to happen.

- [Checkout the PySyft tutorials](https://github.com/OpenMined/PySyft/tree/master/examples/tutorials)


### Join our Slack!

The best way to keep up to date on the latest advancements is to join our community! 

- [Join slack.openmined.org](http://slack.openmined.org)

### Join a Code Project!

The best way to contribute to our community is to become a code contributor! If you want to start "one off" mini-projects, you can go to PySyft GitHub Issues page and search for issues marked `Good First Issue`.

- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

- [Donate through OpenMined's Open Collective Page](https://opencollective.com/openmined)